In [30]:
# import libraries
import requests
from bs4 import BeautifulSoup
import time
import datetime
import pandas as pd
import csv

import smtplib              # send emails

In [50]:
# connect to the website
URL = 'https://www.amazon.com/Kindle-Paperwhite-Essentials-Bundle-including/dp/B09FBYNC2J/ref=sr_1_14?crid=HOMW2YHJEAW2&keywords=e-readers&psr=EY17&qid=1669308826&s=black-friday&sprefix=e-readers%2Cblack-friday%2C196&sr=1-14'

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 YaBrowser/22.11.0.2419 Yowser/2.5 Safari/537.36",
           "Accept-Encoding":"gzip, deflate",
           "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
           "DNT":"1",
           "Connection":"close", 
           "Upgrade-Insecure-Requests":"1"}

wanted_price = 150                        # variable for check_price function

page = requests.get(URL, headers=headers)

In [37]:
soup1 = BeautifulSoup(page.content, "html.parser")
soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

title = soup2.find(id='productTitle').get_text().strip()
title = title[: title.find(',')]
price = soup2.find(id='price_inside_buybox').get_text().strip()[1:]
today =datetime.date.today()

In [38]:
header = ['title', 'price', 'date']
data = [title, price, today]
with open('amazon_web_scraper_dataset.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f, dialect='excel')
    writer.writerow(header)        #inserting the header
    writer.writerow(data)          # inserting the data
    

In [42]:
df = pd.read_csv('amazon_web_scraper_dataset.csv')
df

,title,price,date
0,Kindle Paperwhite Essentials Bundle including ...,126.97,2022-11-24


In [51]:
def check_price(URL, wanted_price):
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 YaBrowser/22.11.0.2419 Yowser/2.5 Safari/537.36",
    "Accept-Encoding":"gzip, deflate",
    "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT":"1",
    "Connection":"close", 
    "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)
    
    soup1 = BeautifulSoup(page.content, "html.parser")
    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    title = soup2.find(id='productTitle').get_text().strip()
    title = title[: title.find(',')]
    
    price = float(soup2.find(id='price_inside_buybox').get_text().strip()[1:])
    
    today = datetime.date.today()
    
    header = ['title', 'price', 'date']
    data = [title, price, today]
    
    # appending data to the csv
    with open('amazon_web_scraper_dataset.csv', 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f, dialect='excel')        
        writer.writerow(data) 
        
    # sending mails
    if price < wanted_price:
        send_mail()

In [47]:
while(True):
    check_price()
    time.sleep(86400)        # check price every day 

KeyboardInterrupt: 

In [48]:
df = pd.read_csv('amazon_web_scraper_dataset.csv')
df

,title,price,date
0,Kindle Paperwhite Essentials Bundle including ...,126.97,2022-11-24
1,Kindle Paperwhite Essentials Bundle including ...,126.97,2022-11-24
2,Kindle Paperwhite Essentials Bundle including ...,126.97,2022-11-24
3,Kindle Paperwhite Essentials Bundle including ...,126.97,2022-11-24
4,Kindle Paperwhite Essentials Bundle including ...,126.97,2022-11-24
5,Kindle Paperwhite Essentials Bundle including ...,126.97,2022-11-24
6,Kindle Paperwhite Essentials Bundle including ...,126.97,2022-11-24


In [49]:
# send mail when a price hits below a certain level

def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.starttls()
    server.ehlo()
    server.login('learning0smtplib@gmail.com', '01081986d')
    
    message = f"The product that you want is now below {wanted_price}!"
    server.sendmail('learning0smtplib@gmail.com', message)